<h2 style="color:blue" align="center">Handwritten digits classification using neural network</h2>

<img src="Figures/MNIST-Handwritten digits.png" />

#### Import the necessary libraries

In [18]:
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time
import math

from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger

#### Load and split MNIST for training and testing

In [19]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

#### Flattening

In [20]:
X_train_flattened = X_train.reshape(len(X_train), 28 * 28)
X_test_flattened = X_test.reshape(len(X_test),  28 * 28) 

#### Normalize the input dataset

In [21]:
X_train_normalized = X_train_flattened / 255
X_test_normalized = X_test_flattened / 255

##### HM model

In [113]:
np.random.seed(3)
def get_model(): 
    model = keras.Sequential([
        # input layer 784 neurons to first hidden layer with 64 neurons
        keras.layers.Dense(64, input_shape = (784,), activation='relu'), 
        # first hidden layer to second hidden layer
        keras.layers.Dense(64, activation='relu'),  
        # Output layer with 10 neurons
        keras.layers.Dense(10, activation='softmax')
    ])
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.01)
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6)
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6, nesterov=True)
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.6)
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    
    model.compile(
        optimizer = opti,
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    return model

In [84]:
class CustomCallback(keras.callbacks.Callback):  
    iteration = 0
    batch_size = 60000
    training_set_size = 0
    update_every_fold = 0
    initial_weights = 0
    previous_weights = 0
    call_hm = 0
    call_am = 0 
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm) 
        #Determining number of updates every epoch
        self.training_set_size = X_train_normalized.shape[0]
        self.update_every_fold = self.fold_calc(self.batch_size, self.training_set_size)
    
         
    def on_epoch_begin(self, epoch,  logs=None): 
        self.iteration = 1   
        
    def on_train_batch_end(self, batch, logs=None): 
        if self.iteration%self.update_every_fold == 0:  
            counter = 0
            num_layers = len(model_hm.layers)  
            current_weights = model_hm.get_weights()
            current_weights = np.array(current_weights,dtype=object)        

            for i in range(num_layers):  
                # Harmonic mean based update
                current_weights[counter] = self.call_hm(self.previous_weights[counter], current_weights[counter])
                counter = counter + 2

            updated = current_weights.tolist()   
            model_hm.set_weights(updated)
            self.previous_weights = current_weights
        self.iteration = self.iteration + 1  
    
    #def on_epoch_end(self, epoch, logs={}):
      # print(model.loss)

    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2  
        
    def fold_calc(self, batch_size, training_set_size): 
        total_fold = int(math.ceil(math.log2(training_set_size)))
        #print("total_fold   :", total_fold) 
        num_batch = math.ceil(training_set_size / batch_size)
        #print("num_batch   :", num_batch)
        batch_fold = int(math.floor(math.log2(batch_size)))
        #print("batch_fold   :", batch_fold)
        update_every_fold = (total_fold-batch_fold)    
        #print("update_every_fold   :", update_every_fold)
        num_updates = num_batch/update_every_fold
        #print("num_updates   :", num_updates)
        return update_every_fold

#### Training

In [85]:
csv_logger = CSVLogger('log.csv', append=False, separator=',')

General method

In [114]:
model_wihtout_hm = get_model() 
st = time.time()
model_wihtout_hm.fit(X_train_normalized, y_train, epochs = 500, verbose=1, callbacks=[csv_logger], batch_size=60000) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/500
1/1 [==============================] - 1s 684ms/step - loss: 2.3550 - accuracy: 0.0972
Epoch 2/500
1/1 [==============================] - 0s 171ms/step - loss: 2.2817 - accuracy: 0.1490
Epoch 3/500
1/1 [==============================] - 0s 161ms/step - loss: 2.2207 - accuracy: 0.1964
Epoch 4/500
1/1 [==============================] - 0s 157ms/step - loss: 2.1662 - accuracy: 0.2542
Epoch 5/500
1/1 [==============================] - 0s 159ms/step - loss: 2.1137 - accuracy: 0.3266
Epoch 6/500
1/1 [==============================] - 0s 156ms/step - loss: 2.0602 - accuracy: 0.4044
Epoch 7/500
1/1 [==============================] - 0s 185ms/step - loss: 2.0045 - accuracy: 0.4707
Epoch 8/500
1/1 [==============================] - 0s 191ms/step - loss: 1.9461 - accuracy: 0.5227
Epoch 9/500
1/1 [==============================] - 0s 170ms/step - loss: 1.8852 - accuracy: 0.5613
Epoch 10/500
1/1 [==============================] - 0s 171ms/step - loss: 1.8223 - accuracy: 0.5904
Epoch 11/

1/1 [==============================] - 0s 170ms/step - loss: 0.2635 - accuracy: 0.9255
Epoch 84/500
1/1 [==============================] - 0s 167ms/step - loss: 0.2614 - accuracy: 0.9262
Epoch 85/500
1/1 [==============================] - 0s 158ms/step - loss: 0.2594 - accuracy: 0.9267
Epoch 86/500
1/1 [==============================] - 0s 164ms/step - loss: 0.2574 - accuracy: 0.9274
Epoch 87/500
1/1 [==============================] - 0s 164ms/step - loss: 0.2554 - accuracy: 0.9279
Epoch 88/500
1/1 [==============================] - 0s 153ms/step - loss: 0.2535 - accuracy: 0.9285
Epoch 89/500
1/1 [==============================] - 0s 161ms/step - loss: 0.2516 - accuracy: 0.9292
Epoch 90/500
1/1 [==============================] - 0s 173ms/step - loss: 0.2498 - accuracy: 0.9298
Epoch 91/500
1/1 [==============================] - 0s 162ms/step - loss: 0.2480 - accuracy: 0.9305
Epoch 92/500
1/1 [==============================] - 0s 154ms/step - loss: 0.2462 - accuracy: 0.9310
Epoch 93/500


1/1 [==============================] - 0s 172ms/step - loss: 0.1129 - accuracy: 0.9678
Epoch 246/500
1/1 [==============================] - 0s 170ms/step - loss: 0.1124 - accuracy: 0.9679
Epoch 247/500
1/1 [==============================] - 0s 179ms/step - loss: 0.1119 - accuracy: 0.9680
Epoch 248/500
1/1 [==============================] - 0s 158ms/step - loss: 0.1115 - accuracy: 0.9681
Epoch 249/500
1/1 [==============================] - 0s 165ms/step - loss: 0.1110 - accuracy: 0.9682
Epoch 250/500
1/1 [==============================] - 0s 165ms/step - loss: 0.1105 - accuracy: 0.9683
Epoch 251/500
1/1 [==============================] - 0s 158ms/step - loss: 0.1101 - accuracy: 0.9685
Epoch 252/500
1/1 [==============================] - 0s 182ms/step - loss: 0.1096 - accuracy: 0.9686
Epoch 253/500
1/1 [==============================] - 0s 166ms/step - loss: 0.1091 - accuracy: 0.9688
Epoch 254/500
1/1 [==============================] - 0s 166ms/step - loss: 0.1087 - accuracy: 0.9689
Epoc

1/1 [==============================] - 0s 173ms/step - loss: 0.0598 - accuracy: 0.9837
Epoch 408/500
1/1 [==============================] - 0s 174ms/step - loss: 0.0596 - accuracy: 0.9838
Epoch 409/500
1/1 [==============================] - 0s 163ms/step - loss: 0.0594 - accuracy: 0.9837
Epoch 410/500
1/1 [==============================] - 0s 155ms/step - loss: 0.0592 - accuracy: 0.9838
Epoch 411/500
1/1 [==============================] - 0s 165ms/step - loss: 0.0589 - accuracy: 0.9839
Epoch 412/500
1/1 [==============================] - 0s 162ms/step - loss: 0.0587 - accuracy: 0.9840
Epoch 413/500
1/1 [==============================] - 0s 167ms/step - loss: 0.0585 - accuracy: 0.9841
Epoch 414/500
1/1 [==============================] - 0s 165ms/step - loss: 0.0583 - accuracy: 0.9841
Epoch 415/500
1/1 [==============================] - 0s 169ms/step - loss: 0.0581 - accuracy: 0.9842
Epoch 416/500
1/1 [==============================] - 0s 159ms/step - loss: 0.0579 - accuracy: 0.9843
Epoc

HM based method

In [115]:
model_hm = get_model() 
st = time.time()
model_hm.fit(X_train_normalized, y_train, epochs = 500, verbose=1, callbacks=[CustomCallback(),csv_logger], batch_size=60000) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/500
1/1 [==============================] - 1s 738ms/step - loss: 2.3671 - accuracy: 0.0896
Epoch 2/500
1/1 [==============================] - 0s 196ms/step - loss: 2.2358 - accuracy: 0.1538
Epoch 3/500
1/1 [==============================] - 0s 210ms/step - loss: 2.1321 - accuracy: 0.2883
Epoch 4/500
1/1 [==============================] - 0s 203ms/step - loss: 2.0436 - accuracy: 0.3796
Epoch 5/500
1/1 [==============================] - 0s 197ms/step - loss: 1.9576 - accuracy: 0.4417
Epoch 6/500
1/1 [==============================] - 0s 231ms/step - loss: 1.8677 - accuracy: 0.4950
Epoch 7/500
1/1 [==============================] - 0s 222ms/step - loss: 1.7729 - accuracy: 0.5450
Epoch 8/500
1/1 [==============================] - 0s 213ms/step - loss: 1.6753 - accuracy: 0.5901
Epoch 9/500
1/1 [==============================] - 0s 251ms/step - loss: 1.5766 - accuracy: 0.6288
Epoch 10/500
1/1 [==============================] - 0s 236ms/step - loss: 1.4787 - accuracy: 0.6647
Epoch 11/

1/1 [==============================] - 0s 208ms/step - loss: 0.2225 - accuracy: 0.9369
Epoch 84/500
1/1 [==============================] - 0s 208ms/step - loss: 0.2206 - accuracy: 0.9373
Epoch 85/500
1/1 [==============================] - 0s 198ms/step - loss: 0.2187 - accuracy: 0.9378
Epoch 86/500
1/1 [==============================] - 0s 201ms/step - loss: 0.2168 - accuracy: 0.9383
Epoch 87/500
1/1 [==============================] - 0s 192ms/step - loss: 0.2149 - accuracy: 0.9388
Epoch 88/500
1/1 [==============================] - 0s 227ms/step - loss: 0.2131 - accuracy: 0.9392
Epoch 89/500
1/1 [==============================] - 0s 222ms/step - loss: 0.2112 - accuracy: 0.9397
Epoch 90/500
1/1 [==============================] - 0s 208ms/step - loss: 0.2095 - accuracy: 0.9402
Epoch 91/500
1/1 [==============================] - 0s 199ms/step - loss: 0.2077 - accuracy: 0.9409
Epoch 92/500
1/1 [==============================] - 0s 209ms/step - loss: 0.2060 - accuracy: 0.9415
Epoch 93/500


1/1 [==============================] - 0s 214ms/step - loss: 0.0794 - accuracy: 0.9779
Epoch 246/500
1/1 [==============================] - 0s 215ms/step - loss: 0.0789 - accuracy: 0.9780
Epoch 247/500
1/1 [==============================] - 0s 203ms/step - loss: 0.0785 - accuracy: 0.9781
Epoch 248/500
1/1 [==============================] - 0s 193ms/step - loss: 0.0781 - accuracy: 0.9783
Epoch 249/500
1/1 [==============================] - 0s 199ms/step - loss: 0.0777 - accuracy: 0.9784
Epoch 250/500
1/1 [==============================] - 0s 205ms/step - loss: 0.0773 - accuracy: 0.9785
Epoch 251/500
1/1 [==============================] - 0s 216ms/step - loss: 0.0769 - accuracy: 0.9786
Epoch 252/500
1/1 [==============================] - 0s 197ms/step - loss: 0.0765 - accuracy: 0.9787
Epoch 253/500
1/1 [==============================] - 0s 192ms/step - loss: 0.0761 - accuracy: 0.9788
Epoch 254/500
1/1 [==============================] - 0s 198ms/step - loss: 0.0757 - accuracy: 0.9790
Epoc

1/1 [==============================] - 0s 223ms/step - loss: 0.0349 - accuracy: 0.9915
Epoch 408/500
1/1 [==============================] - 0s 233ms/step - loss: 0.0347 - accuracy: 0.9915
Epoch 409/500
1/1 [==============================] - 0s 202ms/step - loss: 0.0345 - accuracy: 0.9916
Epoch 410/500
1/1 [==============================] - 0s 195ms/step - loss: 0.0344 - accuracy: 0.9916
Epoch 411/500
1/1 [==============================] - 0s 205ms/step - loss: 0.0342 - accuracy: 0.9916
Epoch 412/500
1/1 [==============================] - 0s 202ms/step - loss: 0.0340 - accuracy: 0.9918
Epoch 413/500
1/1 [==============================] - 0s 212ms/step - loss: 0.0339 - accuracy: 0.9916
Epoch 414/500
1/1 [==============================] - 0s 217ms/step - loss: 0.0337 - accuracy: 0.9918
Epoch 415/500
1/1 [==============================] - 0s 188ms/step - loss: 0.0335 - accuracy: 0.9919
Epoch 416/500
1/1 [==============================] - 0s 195ms/step - loss: 0.0334 - accuracy: 0.9919
Epoc

###### Testing the model

General method

In [353]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1304 - accuracy: 0.9758


[0.13041307032108307, 0.9757999777793884]

HM based method

In [354]:
model_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1261 - accuracy: 0.9767


[0.1261226236820221, 0.9767000079154968]